In [1]:
import simpy
import random
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [48]:
def patient(env,i,p_length_of_stay):
    # add 1 to bed count while the patient is in the hospital
    glob_variables.patient_count+=1
    # patient takes up 1 bed=
    with beds.request() as req:
        yield req
        # trigger event when the patient is better
        yield env.timeout(p_length_of_stay)
    # reduce bed count after patient leaves
    glob_variables.patient_count-=1

def new_patient(env,inter_arrival_time,length_of_stay):
    i = 0
    while True:
        i+=1
        # patient length of stay is drawn from exponential distribution
        p_length_of_stay = random.expovariate(1/length_of_stay)
        # create a new patient
        p = patient(env,i,p_length_of_stay)
        # start patient process
        env.process(p)
        # define next patient based on inter arrival time
        next_p = random.expovariate(1/inter_arrival_time)
        # trigger event when new patient comes
        yield env.timeout(next_p)

In [49]:
def check(env, delay):
    yield env.timeout(delay)
    while True:
        glob_variables.check_times.append(env.now)
        glob_variables.check_beds.append(beds.count)
        glob_variables.check_untreated.append(glob_variables.patient_count-beds.count)
        yield env.timeout(glob_variables.check_intervals)

In [73]:
class glob_variables:
    patient_count = 0
    mean_inter_arrival_time = .1
    mean_length_of_stay = 21
    max_bed_capacity = 100
    sim_duration = 600
    check_times = []
    check_beds = []
    check_untreated = []
    check_intervals = 7
    delay = 7

In [74]:
env = simpy.Environment()
beds = simpy.Resource(env, capacity = glob_variables.max_bed_capacity)
env.process(new_patient(env,glob_variables.mean_inter_arrival_time,glob_variables.mean_length_of_stay))
env.process(check(env,glob_variables.delay))
env.run(until=glob_variables.sim_duration)

In [75]:
Hospital_Ut = pd.DataFrame()
Hospital_Ut["Days Since Outbreak"] = glob_variables.check_times
Hospital_Ut["Bed Count"] = glob_variables.check_beds
Hospital_Ut["Untreated Patients"] = glob_variables.check_untreated

In [76]:
Hospital_Ut

,Days Since Outbreak,Bed Count,Untreated Patients
0,7,63,0
1,14,100,11
2,21,100,53
3,28,100,85
4,35,100,117
...,...,...,...
80,567,100,2934
81,574,100,2988
82,581,100,3029
83,588,100,3048


In [154]:
def plot_graph(x,y):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.plot(Hospital_Ut[x],Hospital_Ut[y])
    ax.set_xlabel(str(x))
    ax.set_ylabel(str(y))
    plt.xlim(0,glob_variables.sim_duration)
    plt.title("{} vs. {}y vs. x,fontweight = "bold")